# RepoHelper Notebook Controller

Use this notebook to steer the Microsoft Agent Framework RepoHelper agent that runs against your local LM Studio endpoint.

1. Ensure LM Studio is serving an OpenAI-compatible API.
2. Run the setup cells below.
3. Edit the task prompt in the last cell and execute it to collect the structured `Patch` response.

In [1]:
import os

os.environ.setdefault("OPENAI_BASE_URL", "http://127.0.0.1:1234/v1")
os.environ.setdefault("OPENAI_API_KEY", "lm-studio")
os.environ.setdefault("OPENAI_CHAT_MODEL_ID", "qwen/qwen3-coder-30b")

print("Endpoint:", os.environ["OPENAI_BASE_URL"]) 
print("Model:", os.environ["OPENAI_CHAT_MODEL_ID"])

Endpoint: http://127.0.0.1:1234/v1
Model: qwen/qwen3-coder-30b


In [ ]:
#!pip install agent-framework --pre
#!pip install --pre agent-framework-core


In [2]:
import asyncio
from agent import (
    Patch,
    create_repo_helper_agent,
    get_repo_helper_agent,
    run_repo_helper,
)

# Reuse the cached agent so every cell call hits the same Microsoft Agent Framework session
repo_helper = get_repo_helper_agent()
repo_helper

## Run tasks

Update the `task` text and re-run the cell to drive the agent. Use `await` to integrate with Jupyter's event loop without extra boilerplate.

In [3]:
task = """\
Read agent.py and suggest a minor improvement. Provide the change as a Patch diff.
"""

result: Patch = await run_repo_helper(task, agent=repo_helper)
result_dict = result.dict()
print("Summary:", result_dict["summary"])
print("Filename:", result_dict["filename"])
print("Diff:\n", result_dict["diff"])
result_dict

Summary: I need to see the content of agent.py to provide a meaningful improvement suggestion. Please share the code from this file.
Filename: agent.py
Diff:
 


/var/folders/sz/qdyj3ffd5rxg1vjk8d82pkkm0000gp/T/ipykernel_22908/77903180.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_dict = result.dict()


{'filename': 'agent.py',
 'summary': 'I need to see the content of agent.py to provide a meaningful improvement suggestion. Please share the code from this file.',
 'diff': ''}